In [ ]:
from random import seed
from random import randrange

def random_algorithm(train, test):
  output_values = [row[-1] for row in train]
  unique = list(set(output_values))
  predicted = list()
  for _ in test:
    index = randrange(len(unique))
    predicted.append(unique[index])
  return predicted

In [ ]:
seed(1)
train = [[0],[1],[0],[1],[0],[1]]
test = [[None],[None],[None],[None]]
predictions = random_algorithm(train,test)
print(predictions)

[0, 0, 1, 0]


In [ ]:

k = [[0],[1],[2],[3],[4],[5]]
for i in k:
  s = list(k)
  s.remove(i)
  s=sum(s,[])
  print(s)

[1, 2, 3, 4, 5]
[0, 2, 3, 4, 5]
[0, 1, 3, 4, 5]
[0, 1, 2, 4, 5]
[0, 1, 2, 3, 5]
[0, 1, 2, 3, 4]


In [ ]:
def covariance(x,mean_x,y,mean_y):
  covar = 0.0
  for i in range(len(x)):
    covar += (x[i]-mean_x)*(y[i]-mean_y)
  return covar
def mean(values):
  return sum(values)/float(len(values))

dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]
x = [row[0] for row in dataset]
y = [row[1] for row in dataset]
mean_x, mean_y = mean(x), mean(y)
covar = covariance(x,mean_x,y,mean_y)
print(covar)

8.0


In [ ]:
def variance(values,mean):
  return sum([(x-mean)**2 for x in values])
def coefficients(dataset):
  x = [row[0] for row in dataset]
  y = [row[1] for row in dataset]
  mean_x, mean_y = mean(x), mean(y)
  b1 = covariance(x,mean_x,y,mean_y)/variance(x,mean_x)
  b0 = mean_y - b1*mean_x
  return [b0,b1]
dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]
b0,b1 = coefficients(dataset)
print("{} x X + {} = Y".format(b1,b0))


NameError: ignored

In [ ]:
def predict(row, coefficients):
  yhat = coefficients[0]
  for i in range(len(row)-1):
    yhat += coefficients[i+1] * row[i]
  return yhat

In [ ]:
dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]
coef = [0.4,0.8]
for row in dataset:
  yhat = predict(row,coef)
  print("expected = %.3f , predicted = %.3f"%(row[-1],yhat))

expected = 1.000 , predicted = 1.200
expected = 3.000 , predicted = 2.000
expected = 3.000 , predicted = 3.600
expected = 2.000 , predicted = 2.800
expected = 5.000 , predicted = 4.400


In [ ]:
def coefficients_sgd(train,l_rate,n_epoch):
  coef = [0.0 for i in range(len(train[0]))]
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      yhat = predict(row,coef)
      error = yhat - row[-1]
      sum_error += error**2
      coef[0] = coef[0] - l_rate * error
      for i in range(len(row)-1):
        coef[i+1] = coef[i+1] - l_rate * error*row[i]
    print("epoch %d, lrate = %.3f, error = %.3f"%(epoch,l_rate,sum_error))
  return coef


In [ ]:
dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]
l_rate=0.001
n_epoch=50
coef=coefficients_sgd(dataset,l_rate,n_epoch)
print(coef)

epoch 0, lrate = 0.001, error = 46.236, coef = [0.014, 0.049]
epoch 1, lrate = 0.001, error = 41.305, coef = [0.027, 0.095]
epoch 2, lrate = 0.001, error = 36.930, coef = [0.039, 0.139]
epoch 3, lrate = 0.001, error = 33.047, coef = [0.051, 0.180]
epoch 4, lrate = 0.001, error = 29.601, coef = [0.061, 0.218]
epoch 5, lrate = 0.001, error = 26.543, coef = [0.072, 0.255]
epoch 6, lrate = 0.001, error = 23.830, coef = [0.081, 0.289]
epoch 7, lrate = 0.001, error = 21.422, coef = [0.090, 0.321]
epoch 8, lrate = 0.001, error = 19.285, coef = [0.099, 0.351]
epoch 9, lrate = 0.001, error = 17.389, coef = [0.107, 0.380]
epoch 10, lrate = 0.001, error = 15.706, coef = [0.115, 0.407]
epoch 11, lrate = 0.001, error = 14.213, coef = [0.122, 0.432]
epoch 12, lrate = 0.001, error = 12.888, coef = [0.129, 0.456]
epoch 13, lrate = 0.001, error = 11.712, coef = [0.135, 0.479]
epoch 14, lrate = 0.001, error = 10.668, coef = [0.141, 0.500]
epoch 15, lrate = 0.001, error = 9.742, coef = [0.147, 0.520]
epo

In [41]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

def load_csv(filename):
  dataset = list()
  with open(filename,'r') as file:
    csv_reader = reader(file)
    next(csv_reader)
    for row in csv_reader:
      if not row:
        continue
      dataset.append(row[0].split(';'))
  return dataset

def str_column_to_float(dataset,column):
  for row in dataset:
    row[column]=float(row[column].strip())

def dataset_minmax(dataset):
  minmax = list()
  for i in range(len(dataset[0])):
    col_values = [row[i] for row in dataset]
    value_min = min(col_values)
    value_max = max(col_values)
    minmax.append([value_min,value_max])
  return minmax

def normalize_dataset(dataset, minmax):
  for row in dataset:
    for i in range(len(row)):
      row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
def cross_validation_split(dataset,n_folds):
  dataset_split = list()
  dataset_copy = list(dataset)
  fold_size = int(len(dataset)/n_folds)
  for _ in range(n_folds):
    fold = list()
    while len(fold) < fold_size:
      index = randrange(len(dataset_copy))
      fold.append(dataset_copy.pop(index))
    dataset_split.append(fold)
  return dataset_split

def rmse_metric(actual,predicted):
  sum_error = 0.0
  for i in range(len(actual)):
    prediction_error = predicted[i] - actual[i]
    sum_error += (prediction_error**2)
  mean_error = sum_error/float(len(actual))
  return sqrt(mean_error)

def evaluate_algorithm(dataset,algorithm,n_folds,*args):
  folds = cross_validation_split(dataset,n_folds)
  scores = list()
  for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set,[])
    test_set = list()
    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      row_copy[-1] = None
    predicted = algorithm(train_set,test_set,*args)
    actual = [row[-1] for row in fold]
    rmse = rmse_metric(actual,predicted)
    scores.append(rmse)
  return scores

def linear_regression_sgd(train,test,l_rate,n_epoch):
  predictions = list()
  coef = coefficients_sgd(train,l_rate,n_epoch)
  for row in test:
    yhat = predict(row,coef)
    predictions.append(yhat)
  return(predictions)

seed(1)
filename = "/content/drive/MyDrive/ML_mastery/winequality-white.csv"
dataset = load_csv(filename)
for i in range(len(dataset[0])):
  str_column_to_float(dataset,i)
minmax = dataset_minmax(dataset)
normalize_dataset(dataset,minmax)

n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset,linear_regression_sgd,n_folds,l_rate,n_epoch)

print("Scores = %s"%scores)
print("Mean RMSE : %.3f"%(sum(scores)/float(len(scores))))

NameError: ignored

In [ ]:
def gini_index(groups,classes):
  n_instances = float(sum([len(group) for group in groups]))
  gini =0.0
  for group in groups:
    size = float(len(group))
    if size ==0:
      continue
    score =0.0
    for class_val in classes:
      p = [row[-1] for row in group].count(class_val)/size
      score += p*p
    gini += (1-score)*(size/n_instances)
  return gini


In [7]:
print(gini_index([[[1,1],[1,0]],[[1,1],[1,0]]],[0,1]))

0.5


In [9]:
print(gini_index([[[1,0],[1,0]],[[1,1],[1,1]]],[0,1]))

0.0


In [ ]:
def test_split(index,value,dataset):
  left,right = list(),list()
  for row in dataset:
    if row[index]<value:
      left.append(row)
    else:
      right.append(row)
  return left,right
def get_split(dataset):
  class_values = list(set(row[-1] for row in dataset))
  b_index , b_value,b_score,b_groups = 999,999,999,None
  for index in range(len(dataset[0])-1):
    for row in dataset:
      groups = test_split(index,row[index],dataset)
      gini = gini_index(groups,class_values)
      if gini <b_score:
        b_index,b_value,b_score,b_groups = index,row[index],gini,groups
  return {'index':b_index,'value':b_value,'groups':b_groups}



In [32]:
dataset=[[2.7712,1.7847,0],[1.72,1.16,0],[3.67,2.8,0],[3.96,2.61,0],[2.96,2.21,0],[7.49,3.16,1],[9.02,3.33,1],[7.44,0.47,1],[10.12,3.23,1],[6.64,3.31,1]]


In [ ]:
def to_terminal(group):
  outcomes = [row[-1] for row in group]
  return max(set(outcomes), key = outcomes.count)
  

In [ ]:
def split(node,max_depth,min_size,depth):
  left,right = node['groups']
  del(node['groups'])
  if not left or not right:
    node['left']= node['right'] = to_terminal(left+right)
    return
  if depth>=max_depth:
    node['left'], node['right'] = to_terminal(left),to_terminal(right)
    return
  if len(left) <= min_size:
    node['left'] = to_terminal(left)
  else :
    node['left'] = get_split(left)
    split(node['left'],max_depth,min_size,depth+1)
  if len(right) <= min_size:
    node['right'] = to_terminal(right)
  else :
    node['right']=get_split(right)
    split(node['right'],max_depth,min_size,depth+1)
  


In [ ]:

def build_tree(train,max_depth,min_size):
  root = get_split(train)
  split(root,max_depth,min_size,1)
  return root
def print_tree(node,depth=0):
  if isinstance(node,dict):
    print('%s[X%d < %.3f]'%((depth*'', (node['index']+1), node['value'])))
    print_tree(node['left'],depth+1)
    print_tree(node['right'],depth+1)
  else:
    print("%s[%s]"%((depth*' ', node)))


In [33]:
tree = build_tree(dataset,1,1)
print_tree(tree)

[X1 < 6.640]
 [0]
 [1]


In [34]:
tree = build_tree(dataset,2,1)
print_tree(tree)

[X1 < 6.640]
[X1 < 2.771]
  [0]
  [0]
[X1 < 7.490]
  [1]
  [1]


In [36]:
def predict(node,row):
  if row[node['index']] < node['value']:
    if isinstance(node['left'],dict):
      return predict(node['left'],row)
    else:
      return node['left']
  else:
    if isinstance(node['right'],dict):
      return predict(node['right'],row)
    else:
      return node['right']
  

In [38]:
stump = {"index":0,'right':1,'value':6.642287351,'left':0}
for row in dataset:
  prediction = predict(stump,row)
  print('expected =%d, got=%d'%(row[-1],prediction))

expected =0, got=0
expected =0, got=0
expected =0, got=0
expected =0, got=0
expected =0, got=0
expected =1, got=1
expected =1, got=1
expected =1, got=1
expected =1, got=1
expected =1, got=0


In [45]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

def load_csv(filename):
  dataset = list()
  with open(filename,'r') as file:
    csv_reader = reader(file)
    next(csv_reader)
    for row in csv_reader:
      if not row:
        continue
      dataset.append(row[0].split(';'))
  return dataset
def accuracy_metric(actual,predicted):
  correct=0
  for i in range(len(actual)):
     if actual[i] == predicted[i]:
      correct +=1
  return correct/float(len(actual)) * 100.0

def str_column_to_float(dataset,column):
  for row in dataset:
    row[column]=float(row[column].strip())
def evaluate_algorithm(dataset,algorithm,n_folds,*args):
  folds = cross_validation_split(dataset,n_folds)
  scores = list()
  for fold in folds:
    train_set = list(folds)
    train_set.remove(fold)
    train_set = sum(train_set,[])
    test_set = list()
    for row in fold:
      row_copy = list(row)
      test_set.append(row_copy)
      row_copy[-1] = None
    predicted = algorithm(train_set,test_set,*args)
    actual = [row[-1] for row in fold]
    accur = accuracy_metric(actual,predicted)
    scores.append(accur)
  return scores

In [46]:
def decision_tree(train,test,max_depth,min_size):
  tree = build_tree(train,max_depth,min_size)
  predictions=list()
  for row in test:
    prediction = predict(tree,row)
    predictions.append(prediction)
  return(predictions)

seed(1)
filename = "/content/drive/MyDrive/ML_mastery/winequality-white.csv"
dataset = load_csv(filename)
for i in range(len(dataset[0])):
  str_column_to_float(dataset,i)


n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset,decision_tree,n_folds,max_depth,min_size)

print("Scores = %s"%scores)
print("Mean Accuracy : %.3f"%(sum(scores)/float(len(scores))))

Scores = [55.87334014300307, 54.13687436159347, 51.787538304392235, 52.706843718079675, 53.217568947906024]
Mean Accuracy : 53.544
